In [16]:
import tesserocr
from PIL import Image
import numpy as np

image = Image.open('captcha2.png')
image = image.convert('L')
threshold  = 100
array = np.array(image)
array = np.where(array > threshold ,255, 0)

image = Image.fromarray(array.astype('uint8'))
image.show()
print(tesserocr.image_to_text(image))

a b 3 c



In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from io import BytesIO
import numpy as np
from PIL import Image
import tesserocr
import re
import time

from retrying import retry
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

LOGIN_URL = 'https://captcha7.scrape.center/'
ACCOUTN = 'admin'
PASSWORD = 'admin'
THRESHOLD = 150

def preprocess(image):
    image = image.convert('L')
    array = np.array(image)
    array = np.where(array > THRESHOLD, 255, 0)
    image = Image.fromarray(array.astype('uint8'))
    return image

@retry(stop_max_attempt_number=10, retry_on_result=lambda x:x is False)
def login():
    # browser = webdriver.Chrome()
    browser.get(LOGIN_URL)

    ### 这里应该加一个等待
    # 填入账号和密码
    browser.find_element(By.CSS_SELECTOR, '.username input[type="text"]').send_keys(ACCOUTN)
    time.sleep(1)
    browser.find_element(By.CSS_SELECTOR, '.password input[type="password"]').send_keys(PASSWORD)
    time.sleep(1)

    # 验证码图片识别
    captcha = browser.find_element(By.CSS_SELECTOR, '#captcha')
    image = Image.open(BytesIO(captcha.screenshot_as_png))
    print(type(captcha.screenshot_as_png))
    image = preprocess(image)
    captcha = tesserocr.image_to_text(image)
    captcha = re.sub('[^A-Za-z0-9]', '', captcha)

    # 填入验证码
    browser.find_element(By.CSS_SELECTOR, '.captcha input').send_keys(captcha)
    time.sleep(1)

    # 点击登录按钮
    # browser.find_element(By.CSS_SELECTOR, 'button[type="button"]').click()
    browser.find_element(By.CSS_SELECTOR, '.login').click()

    # 判断是否登录成功
    try:
        WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.XPATH, '//h2[contains(., "登录成功")]')))
        time.sleep(5)
        browser.close()
        return True
    except TimeoutException:
        return False

if __name__ == "__main__":
    browser = webdriver.Chrome()
    login()

<class 'bytes'>
